# Fake News Detection Englsih


> ## Install Requirements

In [ ]:
!pip install farasapy
!pip install tqdm
!pip install gensim
!pip install spacy
!pip install scipy
!pip install wandb
!pip install nltk

> ## Import Requirements

In [ ]:
import pandas as pd 
import re
import string
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time
from pathlib import Path
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import joblib
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from farasa.stemmer import FarasaStemmer
from farasa.segmenter import FarasaSegmenter
import unicodedata
from nltk.stem import WordNetLemmatizer

from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import joblib
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report , confusion_matrix, ConfusionMatrixDisplay

from gensim.models.fasttext import load_facebook_vectors


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input,BatchNormalization
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, BatchNormalization, Dense, Bidirectional



import torch
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

> ## Import Data

In [1]:
df_en =  pd.read_csv('/Cleaned_Data/processed_en_data.csv')

NameError: name 'pd' is not defined

In [ ]:
df_en

In [ ]:
X = df_en['text']
y = df_en['label']

> ##  Vectorization (TFIDF)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X_tfidf = tfidf.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.20 , random_state=42, stratify=y)

> ## Train ML Models

In [ ]:

models = {
    "Logistic Regression": LogisticRegression(),
    "Gradient Boosting": GradientBoostingClassifier(random_state=0),
    "Random Forest": RandomForestClassifier(random_state=0),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "SVM": SVC(probability=True)  
}

def train_and_evaluate_model(name, model, X_train, y_train, X_test, y_test):
    """Train a single model and evaluate its performance"""
    print(f"⏳ Training {name}...")
    start_time = time()
    
    # Train model
    model.fit(X_train, y_train)
    training_time = time() - start_time
    
    # Evaluate
    y_pred = model.predict(X_test)
    
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'training_time': training_time
    }
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    print(f"✅ {name} trained in {training_time:.2f}s")
    print(f"\n🔍 {name} Metrics:")
    print(f"  Training Time: {metrics['training_time']:.2f}s")
    print(f"  Accuracy: {metrics['accuracy']:.4f}")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall: {metrics['recall']:.4f}")
    print(f"  F1 Score: {metrics['f1']:.4f}")
    print("Classification Report:")
    print(report)

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    disp.plot(cmap='Blues')
    plt.title(f"Confusion Matrix - {name}")
    plt.show()


    joblib.dump(model, f"./Models/English/{name}_model.pkl")
    print(f"✅ {name} model saved as {name}_model.pkl")
    print("─" * 50)
    return model, name, metrics, report



results = []
for name, model in models.items():
    result = train_and_evaluate_model(name, model, X_train, y_train, X_test, y_test)
    results.append(result)

> ## Word embedding ( Word2Vec)

Download the model from : 
https://www.kaggle.com/datasets/sugataghosh/google-word2vec

In [ ]:
from gensim.models import KeyedVectors

word2vec = KeyedVectors.load_word2vec_format(
    'path/GoogleNews-vectors-negative300.bin', 
    binary=True
)

In [ ]:
print(f"Vocabulary Size: {len(word2vec.index_to_key)}")
print(f"Vector Size: {word2vec.vector_size}")

In [ ]:
word = "book"  
vector = word2vec[word]  

print(f"Vector for '{word}':\n{vector[:10]}...")

In [ ]:
similar_words = word2vec.most_similar("book", topn=5)  
print("Most similar words to 'book':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

In [ ]:
word1 = "woman"  
word2 = "man"  
similarity = word2vec.similarity(word1, word2)

print(f"Similarity between '{word1}' and '{word2}': {similarity:.4f}")

Find a word that fits the analogy:
* "king" (King) - "Man" (Man) + "Woamn" (Woman) ≈ ???
* (i.e., What is to "Woman" as "King" is to "Man"?)

In [ ]:
analogy_result = word2vec.most_similar(positive=["woman","king"], negative=["man"], topn=1)
print(f"Analogy result (King - Man + Woman): {analogy_result[0]}")

In [ ]:
king_vector = word2vec['king']  
man_vector = word2vec['man']  
woman_vector = word2vec['woman']  

# Perform the word analogy: King - Man + Woman
result_vector = (king_vector - man_vector ) + woman_vector


result = word2vec.similar_by_vector(result_vector, topn=1)

# Output the result
print(f"Resulting word for 'woman + man - king' : {result[0]}")

In [ ]:
analogy_result = word2vec.most_similar(positive=["woman" , "father"], negative=["man"], topn=1)
print(f"Analogy result : {analogy_result[0]}")

In [ ]:
word = "AI"  
if word in word2vec:
    print(f"'{word}' exists in vocabulary.")
else:
    print(f"'{word}' NOT found in vocabulary.")

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_NUM_WORDS=50000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df_en["text"])
sequences = tokenizer.texts_to_sequences(df_en["text"])
word_index = tokenizer.word_index

X = pad_sequences(sequences, maxlen=300)

In [ ]:
vocab_size = len(word_index) + 1
vocab_size

In [ ]:
embedding_dim = word2vec.vector_size

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]


y = df_en['label'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

> ## Train LSTM Model

In [ ]:
model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_length=300,
        trainable=False  
    ),

    Bidirectional(LSTM(256, return_sequences=True)), 
    Dropout(0.4),
    BatchNormalization(),
    LSTM(128),  
    Dropout(0.4),
    Dense(64, activation='relu'), 
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

early_stopping = EarlyStopping(monitor='val_loss',  
                               patience=3,  
                               restore_best_weights=True)  



reduce_lr = ReduceLROnPlateau(monitor='val_loss',  factor=0.2,  patience=2,   min_lr=1e-6,    verbose=1)


history = model.fit(
    X_train, y_train,
    epochs=10,                         
    batch_size=128,                     
    validation_split=0.2,              
    verbose=1,
    callbacks=[early_stopping,  
               reduce_lr
               ]  
)

In [ ]:
model.save("./Models/English/lstm.h5") 
print("✅ Model saved as my_model.h5")

In [ ]:
# Evaluate the model
loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

# Additional metrics
from sklearn.metrics import classification_report
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

In [ ]:
# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid()
plt.show()

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


y_pred_proba = model.predict(X_test)

auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC Score: {auc_score:.4f}")

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC = {auc_score:.4f}")
plt.plot([0, 1], [0, 1], 'k--')  # diagonal
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
def predict_fake_news(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=300)  

    # Predict
    prediction = model.predict(padded)[0][0]
    label = "Fake" if prediction > 0.5 else "Real"

    print(f"Text: {text}")
    print(f"Prediction: {prediction:.4f} → {label}")

predict_fake_news("Trump Said Some INSANELY Racist Stuff Inside The Oval Office, And Witnesses Back It Up")
predict_fake_news("Alabama to certify Democrat Jones winner of Senate election")

> ## Train XLM Roberta Model

In [ ]:


train_df, temp_df = train_test_split(df_en, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Load tokenizer and model
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=len(df_en['label'].unique()))

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['Full Text'], padding="max_length", truncation=True, max_length=300)
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")  
print(f"GPU count: {torch.cuda.device_count()}")       
print(f"Current device: {torch.cuda.current_device()}")  
print(f"Device name: {torch.cuda.get_device_name(0)}")

In [ ]:
import wandb
wandb.login(key="your key here")

In [ ]:
wandb.init(project="FND_project_english")

In [ ]:

training_args = TrainingArguments(
    output_dir="results",
    eval_strategy="epoch",  
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,

    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",              
    logging_strategy="steps",      
    disable_tqdm=False,
    logging_steps=1,
    report_to="all"

)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
# Evaluate the model
test_results = trainer.evaluate(test_dataset)
print(test_results)

predictions, labels, _ = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions), dim=1)
print(classification_report(labels, preds, target_names=[str(i) for i in range(len(df_en['label'].unique()))]))

In [ ]:
model.save_pretrained("./Models/English/prerainmodel")

tokenizer.save_pretrained("./Models/English/prerainmodel")

print("✅ Model and tokenizer saved.")